1: The query retrieves all suppliers who have a `PaymentDays` value greater than the minimum `PaymentDays` in the `Purchasing.Suppliers` table (if it is less than 30) or greater than 30 (if the minimum is 30 or more).


In [ ]:
use WideWorldImporters

SELECT SupplierName, PaymentDays 
FROM Purchasing.Suppliers 
WHERE PaymentDays > (
    SELECT CASE 
        WHEN MIN(PaymentDays) < 30 THEN MIN(PaymentDays) 
        ELSE 30 
    END 
    FROM Purchasing.Suppliers 
    WHERE PaymentDays IS NOT NULL
);


2: **Scalar Subquery to Find Most Expensive Product per Order** :
This query uses a self-contained scalar subquery to find the most expensive product in each order. The subquery calculates the maximum UnitPrice for each OrderID in the Sales.OrderLines table, returning it as a new column in the main query.

In [ ]:
USE WideWorldImporters;

SELECT 
    O.OrderID, 
    (SELECT MAX(UnitPrice) 
     FROM Sales.OrderLines OL 
     WHERE OL.OrderID = O.OrderID) AS MostExpensiveProduct
FROM 
    Sales.Orders O;


3: **Multivalued Subquery to Find Customers with Multiple Order**s : This self-contained multivalued subquery retrieves customers who have placed more than one order. The subquery groups the Orders table by CustomerID, counting the number of orders for each customer and returning only those with multiple orders.

In [ ]:
USE WideWorldImporters;

SELECT 
    CustomerName
FROM 
    Sales.Customers
WHERE 
    CustomerID IN (
        SELECT CustomerID 
        FROM Sales.Orders 
        GROUP BY CustomerID 
        HAVING COUNT(OrderID) > 1
    );


4: **Common Table Expression (CTE) for Top Selling Products :** This query uses a **Common Table Expression (CTE)** named `TopProducts` to calculate the total quantity sold for each product. The main query then joins this CTE with the `Warehouse.StockItems` table to retrieve product names and display them in descending order of quantity sold.

In [ ]:
USE WideWorldImporters;

WITH TopProducts AS (
    SELECT StockItemID, SUM(Quantity) AS TotalSold
    FROM Sales.OrderLines
    GROUP BY StockItemID
)

SELECT 
    SI.StockItemName, 
    TP.TotalSold
FROM 
    Warehouse.StockItems SI
JOIN 
    TopProducts TP ON SI.StockItemID = TP.StockItemID
ORDER BY 
    TP.TotalSold DESC;


5: **Correlated Subquery to Retrieve Previous Order Date for Each Customer :** This query uses a **correlated subquery** to retrieve the previous order date for each customer. For each order in the `Sales.Orders` table, the subquery searches for the most recent earlier order by the same customer.

In [ ]:
USE WideWorldImporters;

SELECT 
    O.CustomerID, 
    O.OrderID, 
    O.OrderDate,
    (SELECT MAX(O2.OrderDate) 
     FROM Sales.Orders O2 
     WHERE O2.CustomerID = O.CustomerID 
     AND O2.OrderDate < O.OrderDate) AS PreviousOrderDate
FROM 
    Sales.Orders O;


6: **Using Running Aggregates to Calculate Running Total of Sales** : This query uses a window function inside a Common Table Expression (CTE) to calculate the running total of sales. It adds the total order value cumulatively as the query progresses through the result set, based on OrderDate.

In [ ]:
USE WideWorldImporters;

WITH RunningTotals AS (
    SELECT 
        O.OrderID, 
        O.OrderDate, 
        SUM(Quantity * UnitPrice) OVER (ORDER BY O.OrderDate ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS RunningTotal
    FROM 
        Sales.Orders O
    JOIN 
        Sales.OrderLines OL ON O.OrderID = OL.OrderID
)

SELECT 
    OrderID, 
    OrderDate, 
    RunningTotal
FROM 
    RunningTotals;


7: **Derived Table with Aggregate to Find Orders with High Quantities :** This query uses a **derived table** to calculate the total quantity for each order. The main query filters the results to show only those orders where the total quantity exceeds 100 units.

In [ ]:
USE WideWorldImporters;

SELECT 
    O.OrderID, 
    OT.TotalQuantity
FROM 
    Sales.Orders O
JOIN 
    (SELECT OrderID, SUM(Quantity) AS TotalQuantity
     FROM Sales.OrderLines
     GROUP BY OrderID) OT
ON 
    O.OrderID = OT.OrderID
WHERE 
    OT.TotalQuantity > 100;


8: **Using the APPLY Operator to Retrieve Most Recent Order for Each Customer :** This query uses the **CROSS APPLY** operator to retrieve the most recent order for each customer. The subquery inside the `APPLY` operator returns the most recent `OrderID` and `OrderDate` for each customer, ordered by `OrderDate` in descending order.

In [ ]:
USE WideWorldImporters;

SELECT 
    C.CustomerName, 
    O.OrderID, 
    O.OrderDate
FROM 
    Sales.Customers C
CROSS APPLY (
    SELECT TOP 1 OrderID, OrderDate
    FROM Sales.Orders O 
    WHERE O.CustomerID = C.CustomerID
    ORDER BY O.OrderDate DESC
) O;


9: **Recursive CTE to Generate a Sequence of Numbers :** This query uses a **recursive CTE** to generate a sequence of numbers from 1 to 10. The recursive CTE starts with `1` and repeatedly increments the value by `1` until it reaches `10`. This kind of query is useful for generating sequences or hierarchies.

In [ ]:
USE WideWorldImporters;

WITH NumberSequence AS (
    SELECT 1 AS Number
    UNION ALL
    SELECT Number + 1 
    FROM NumberSequence
    WHERE Number < 10
)

SELECT 
    Number
FROM 
    NumberSequence;


10: **Common Table Expression (CTE) for Monthly Sales Summary :** This query uses a **CTE** to calculate the total sales per month. The `MonthlySales` CTE groups orders by the month in which they were placed and calculates the total sales. The main query retrieves and orders this summary.

In [ ]:
USE WideWorldImporters;

WITH MonthlySales AS (
    SELECT 
        DATEPART(MONTH, OrderDate) AS OrderMonth, 
        SUM(Quantity * UnitPrice) AS TotalSales
    FROM 
        Sales.OrderLines OL
    JOIN 
        Sales.Orders O ON OL.OrderID = O.OrderID
    GROUP BY 
        DATEPART(MONTH, OrderDate)
)

SELECT 
    OrderMonth, 
    TotalSales
FROM 
    MonthlySales
ORDER BY 
    OrderMonth;
